# First  data

In [29]:
import sys
import os
import time
import datetime
import pandas as pd
import numpy as numpy
from requests import get
from bs4 import BeautifulSoup
from itertools import groupby 

In [30]:
def get_data_second():
    url ="https://bubblezs.wordpress.com/2016/08/20/bollywood-actors-and-their-zodiac-signs-sun-signs/"
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    a =  html_soup.find_all("td",colspan="3", width="327")
    sign = []
    traits =[]
    for i in range(0,len(a),2):
        sign.append(a[i].text)
        traits.append(a[i+1].text)
        
    b = html_soup.find_all("td", width="141")
    celeb_names=[]
    for j in range(len(b)):
            celeb_names.append(b[j].text)
    res = [list(sub) for ele, sub in groupby(celeb_names, key = bool) if ele]
    df = pd.DataFrame({"sign":sign,"traits":traits,"celeb_names":res})
    return df

In [31]:
df2 = get_data_second()

In [ ]:
def get_born_dates(url_list):
    born_dates = []
    for i in url_list:
        ex_url = "https://www.imdb.com"+i
        response = get(ex_url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        _list_1 = html_soup.find_all("div",id="name-born-info")
        try:
            el = _list_1[0].text
            el =  el.replace("\n","").split()
            mon = re.sub("Born:+","",el[0]).strip()
            date = el[1].replace(",","").strip()
            year = el[2].strip()
            born = ",".join([mon,date,year])
            born_dates.append(born)
        except:
            born_dates.append("")
    return born_dates


In [ ]:
def get_data_first():
    images =[]
    ref =[]
    celeb_names =[]
    for i in range(1,3):
        url ="https://www.imdb.com/list/ls068010962/?sort=list_order,asc&mode=detail&page="+str(i)
        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        _list_1 = html_soup.find_all("div",class_="lister-item-image")
        for j in _list_1:
            a=str(j).split("\n")
            b = re.sub("<img alt=+(.*?)>","",a[1])
            c=re.sub("a href=","",b).replace('"' ,"").strip()
            c = c.replace("<","")
            c = c.replace(">","")
            ref.append(c)
        _list_2 = html_soup.find_all("img")
        for k in _list_2:
            if k.get("alt")!="list image":
                a = k.get("src")
                b = k.get("alt")
                celeb_names.append(b)
                images.append(a)
    born_dates = get_born_dates(ref)            
    df = pd.DataFrame({"actor_image":images,"celeb_names":celeb_names,"born day":born_dates})
    return df

In [ ]:
df = get_data_first()

In [ ]:
#df.to_csv("celebrity_2.csv",index=False)

In [11]:
data = pd.read_csv("celebrity_2.csv")

In [12]:
data = data.dropna()
data.drop(data.index[[142]],inplace=True)

In [13]:
data = data.reset_index(drop=True)

In [14]:
data.head()

,actor_image,celeb_names,born day
0,https://m.media-amazon.com/images/M/MV5BMjAwMj...,Aamir Khan,"March,14,1965"
1,https://m.media-amazon.com/images/M/MV5BODI4ND...,Akshay Kumar,"September,9,1967"
2,https://m.media-amazon.com/images/M/MV5BMTY0MD...,Ajay Devgn,"April,2,1969"
3,https://m.media-amazon.com/images/M/MV5BOWUzNj...,Amjad Khan,"November,12,1940"
4,https://m.media-amazon.com/images/M/MV5BNTk1OT...,Amitabh Bachchan,"October,11,1942"


In [15]:
def zodiac_sign(data):
    d = data.split(",")
    day = int(d[1])
    month = d[0].lower()
      
    # checks month and date within the valid range 
    # of a specified zodiac 
    if month == 'december': 
        astro_sign = 'Sagittarius' if (day < 22) else 'capricorn'
          
    elif month == 'january': 
        astro_sign = 'Capricorn' if (day < 20) else 'aquarius'
          
    elif month == 'february': 
        astro_sign = 'Aquarius' if (day < 19) else 'pisces'
          
    elif month == 'march': 
        astro_sign = 'Pisces' if (day < 21) else 'aries'
          
    elif month == 'april': 
        astro_sign = 'Aries' if (day < 20) else 'taurus'
          
    elif month == 'may': 
        astro_sign = 'Taurus' if (day < 21) else 'gemini'
          
    elif month == 'june': 
        astro_sign = 'Gemini' if (day < 21) else 'cancer'
          
    elif month == 'july': 
        astro_sign = 'Cancer' if (day < 23) else 'leo'
          
    elif month == 'august': 
        astro_sign = 'Leo' if (day < 23) else 'virgo'
          
    elif month == 'september': 
        astro_sign = 'Virgo' if (day < 23) else 'libra'
          
    elif month == 'october': 
        astro_sign = 'Libra' if (day < 23) else 'scorpio'
          
    elif month == 'november': 
        astro_sign = 'scorpio' if (day < 22) else 'sagittarius'
    return astro_sign.lower()  

In [16]:
zodiac = []
for i in range(data.shape[0]):
    zodiac.append(zodiac_sign(data.loc[i,"born day"]))

In [17]:
data["zodiac"] = zodiac

In [18]:
data

,actor_image,celeb_names,born day,zodiac
0,https://m.media-amazon.com/images/M/MV5BMjAwMj...,Aamir Khan,"March,14,1965",pisces
1,https://m.media-amazon.com/images/M/MV5BODI4ND...,Akshay Kumar,"September,9,1967",virgo
2,https://m.media-amazon.com/images/M/MV5BMTY0MD...,Ajay Devgn,"April,2,1969",aries
3,https://m.media-amazon.com/images/M/MV5BOWUzNj...,Amjad Khan,"November,12,1940",scorpio
4,https://m.media-amazon.com/images/M/MV5BNTk1OT...,Amitabh Bachchan,"October,11,1942",libra
...,...,...,...,...
162,https://m.media-amazon.com/images/M/MV5BMTI4OW...,Vrajesh Hirjee,"June,16,1971",gemini
163,https://m.media-amazon.com/images/M/MV5BMTQ4Nz...,Waheeda Rehman,"February,3,1938",aquarius
164,https://m.media-amazon.com/images/M/MV5BMzczOG...,Yashpal Sharma,"August,9,1970",leo
165,https://m.media-amazon.com/images/M/MV5BMTg4ND...,Zeenat Aman,"November,19,1951",scorpio


In [32]:
sign = df2["sign"].str.lower()

In [33]:
df2.drop("sign",axis=1,inplace=True)

In [34]:
df2["sign"] = sign

In [35]:
df2

,traits,celeb_names,sign
0,"Active, Demanding, Determined, Effective, Ambi...","[Ajay Devgn, Akshaye Khanna, Arshad Warsi, Emr...",aries
1,"Logical, Patient, Subtle strength, Appreciatio...","[Anushka Sharma, Madhuri Dixit, Manoj Bajpayee...",taurus
2,"Quick witted, smart, Indecision, Inquisitive, ...","[Ameesha Patel, Dimple Kapadia, Kajal Agarwal,...",gemini
3,"Emotion, Diplomatic, Caring, Dedicated, Select...","[Arjun Kapoor, Armaan Malik, Atul Agnihotri, K...",cancer
4,"Ruling, Proud, Warmth, Generosity, Faithful, I...","[Arbaaz Khan, Bhumika Chawla, Dhanush (Tamil),...",leo
5,"Analyzing, Practical, Observation, Thoughtful,...","[Akshay Kumar, Gulshan Grover, Kareena Kapoor,...",virgo
6,"Balance, Justice, Truth, Beauty, Perfection (h...","[Amitabh Bachchan, Chunky Pandey, Dev Anand, D...",libra
7,"Mysterious, Self-Willed, Purposeful, Rational,...","[Aditya Roy Kapur, Aishwarya Rai, Anu Malik, A...",scorpio
8,"Insightful, Lively, Optimistic, Experimental, ...","[Amol Palekar, Arjun Rampal, Boman Irani, Dhar...",sagittarius
9,"Patient, Practical, Ambitious, Disciplined, Ge...","[A.R.Rahman, Aditya Pancholi, Anil Kapoor, Bip...",capricorn


In [45]:
traits=[]
for i in range(data.shape[0]):
    tr = df2[df2["sign"]==data.loc[i,"zodiac"]].loc[:,"traits"].values
    traits.append(tr)

In [46]:
data["traits"] = traits

In [47]:
data

,actor_image,celeb_names,born day,zodiac,traits
0,https://m.media-amazon.com/images/M/MV5BMjAwMj...,Aamir Khan,"March,14,1965",pisces,"[Platonic, Depth, Imagination, Reactive, Sensi..."
1,https://m.media-amazon.com/images/M/MV5BODI4ND...,Akshay Kumar,"September,9,1967",virgo,"[Analyzing, Practical, Observation, Thoughtful..."
2,https://m.media-amazon.com/images/M/MV5BMTY0MD...,Ajay Devgn,"April,2,1969",aries,"[Active, Demanding, Determined, Effective, Amb..."
3,https://m.media-amazon.com/images/M/MV5BOWUzNj...,Amjad Khan,"November,12,1940",scorpio,"[Mysterious, Self-Willed, Purposeful, Rational..."
4,https://m.media-amazon.com/images/M/MV5BNTk1OT...,Amitabh Bachchan,"October,11,1942",libra,"[Balance, Justice, Truth, Beauty, Perfection (..."
...,...,...,...,...,...
162,https://m.media-amazon.com/images/M/MV5BMTI4OW...,Vrajesh Hirjee,"June,16,1971",gemini,"[Quick witted, smart, Indecision, Inquisitive,..."
163,https://m.media-amazon.com/images/M/MV5BMTQ4Nz...,Waheeda Rehman,"February,3,1938",aquarius,"[Knowledge, Humanitarian, Serious, Insightful,..."
164,https://m.media-amazon.com/images/M/MV5BMzczOG...,Yashpal Sharma,"August,9,1970",leo,"[Ruling, Proud, Warmth, Generosity, Faithful, ..."
165,https://m.media-amazon.com/images/M/MV5BMTg4ND...,Zeenat Aman,"November,19,1951",scorpio,"[Mysterious, Self-Willed, Purposeful, Rational..."
